In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression


In [2]:
# Define sMAPE
def smape(y_true, y_pred):
    """
    Computes the symmetric mean absolute percentage error between the true and predicted values.
    
    Parameters:
        y_true (array-like): true values of the target variable.
        y_pred (array-like): predicted values of the target variable.
    
    Returns:
        smape (float): symmetric mean absolute percentage error between y_true and y_pred.
    """
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))


In [3]:
# Load the merged_df_clean dataset
merged_df_clean = pd.read_csv('merged_df_clean.csv')

In [4]:
# List of regressors to evaluate
regressors = [LGBMRegressor(), XGBRegressor(), RandomForestRegressor(), ElasticNet(), Lasso(), Ridge(), LinearRegression()]

In [5]:
# Create a new dataframe to store the results
results_df = pd.DataFrame(columns=['cfips', 'model_name', 'smape'])

In [12]:

# loop through each cfips
for cfips in merged_df_clean['cfips'].unique():

    # filter the data for the current cfips
    cfips_data = merged_df_clean[merged_df_clean['cfips'] == cfips]

    # perform train-test split
    X_train, X_test, y_train, y_test = train_test_split(cfips_data[['pct_bb', 'pct_college', 'active']], cfips_data['microbusiness_density'], test_size=0.2, random_state=42)

    # initialize the models
    lgbm_model = LGBMRegressor()
    xgb_model = XGBRegressor()
    rf_model = RandomForestRegressor()
    en_model = ElasticNet()
    lasso_model = Lasso()
    ridge_model = Ridge()
    lr_model = LinearRegression()

    # perform cross-validation on each model
    models = [('LGBM', lgbm_model), ('XGB', xgb_model), ('RandomForest', rf_model), ('ElasticNet', en_model), ('Lasso', lasso_model), ('Ridge', ridge_model), ('LinearRegression', lr_model)]
    smape_results = []
    for name, model in models:
        kf = KFold(n_splits=7, shuffle=True, random_state=42)
        smape_scores = []
        for train_index, val_index in kf.split(X_train):
            X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]
            model.fit(X_train_kf, y_train_kf)
            y_pred_kf = model.predict(X_val_kf)
            smape_score = smape(y_val_kf, y_pred_kf)
            smape_scores.concat(smape_score)
        mean_smape = np.mean(smape_scores)
        smape_results.concat((name, mean_smape))

    # select the best model based on the mean smape
    best_model = min(smape_results, key=lambda x: x[1])

    # evaluate the best model on the test set
    model = next(filter(lambda x: x[0] == best_model[0], models))[1]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    smape_score = smape(y_test, y_pred)

    # save the results to the dataframe
    results_df = results_df.concat({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)


/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/3864465240.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

ValueError: Cannot have number of splits n_splits=7 greater than the number of samples: n_samples=4.

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [12], in <cell line: 2>()
     23 kf = KFold(n_splits=7, shuffle=True, random_state=42)
     24 smape_scores = []
---> 25 for train_index, val_index in kf.split(X_train):
     26     X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]
     27     y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:333, in _BaseKFold.split(self, X, y, groups)
    331 n_samples = _num_samples(X)
    332 if self.n_splits > n_samples:
--> 333     raise ValueError(
    334         (
    335             "Cannot have number of splits n_splits={0} greater"
    336             " than the number of samples: n_samples={1}."
    337         ).format(self.n_splits, n_samples)
    338     )
    340 for train, test in super().split(X, y, groups):
    341     yield train, test

ValueError: Cannot have number of splits n_splits=7 greater than the number of samples: n_samples=4.


In [13]:
# create an empty dataframe to store the results
results_df = pd.DataFrame(columns=['cfips', 'best_model', 'best_smape'])

# loop through each cfips
for cfips in merged_df_clean['cfips'].unique():

    # filter the data for the current cfips
    cfips_data = merged_df_clean[merged_df_clean['cfips'] == cfips]
    
    # skip CFIPS with less than 2 samples
    if len(cfips_data) < 2:
        continue
    
    # perform train-test split
    X_train, X_test, y_train, y_test = train_test_split(cfips_data[['pct_bb', 'pct_college', 'active']], cfips_data['microbusiness_density'], test_size=0.2, random_state=42)

    # skip CFIPS with not enough samples for KFold cross-validation
    if len(X_train) < 7:
        # train and evaluate each model on the entire training set
        lgbm_model = LGBMRegressor()
        xgb_model = XGBRegressor()
        rf_model = RandomForestRegressor()
        en_model = ElasticNet()
        lasso_model = Lasso()
        ridge_model = Ridge()
        lr_model = LinearRegression()

        models = [('LGBM', lgbm_model), ('XGB', xgb_model), ('RandomForest', rf_model), ('ElasticNet', en_model), ('Lasso', lasso_model), ('Ridge', ridge_model), ('LinearRegression', lr_model)]
        smape_results = []
        for name, model in models:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            smape_score = smape(y_test, y_pred)
            smape_results.append((name, smape_score))

        # select the best model based on the test smape
        best_model = min(smape_results, key=lambda x: x[1])

    else:
        # perform cross-validation on each model
        lgbm_model = LGBMRegressor()
        xgb_model = XGBRegressor()
        rf_model = RandomForestRegressor()
        en_model = ElasticNet()
        lasso_model = Lasso()
        ridge_model = Ridge()
        lr_model = LinearRegression()

        models = [('LGBM', lgbm_model), ('XGB', xgb_model), ('RandomForest', rf_model), ('ElasticNet', en_model), ('Lasso', lasso_model), ('Ridge', ridge_model), ('LinearRegression', lr_model)]
        smape_results = []
        for name, model in models:
            kf = KFold(n_splits=7, shuffle=True, random_state=42)
            smape_scores = []
            for train_index, val_index in kf.split(X_train):
                X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]
                y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]
                model.fit(X_train_kf, y_train_kf)
                y_pred_kf = model.predict(X_val_kf)
                smape_score = smape(y_val_kf, y_pred_kf)
                smape_scores.append(smape_score)
            mean_smape = np.mean(smape_scores)
            smape_results.append((name, mean_smape))

        # select the best model based on the mean sm
    # select the best model based on the mean smape
    best_model = min(smape_results, key=lambda x: x[1])

    # evaluate the best model on the test set
    model = next(filter(lambda x: x[0] == best_model[0], models))[1]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    smape_score = smape(y_test, y_pred)

    # save the results to the dataframe
    results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)


/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

...

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': best_model[1]}, ignore_index=True)
/var/folders/_f/hs08xj6n67n6pm8khvn7t4ww0000gn/T/ipykernel_40415/2700348701.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'cfips': cfips, 'best_model': best_model[0], 'best_smape': bes

In [16]:
print(results_df)

      cfips        best_model  best_smape
0      1001  LinearRegression    0.016664
1      1003  LinearRegression    0.093293
2      1005             Ridge    0.528527
3      1007             Ridge    0.046705
4      1009             Ridge    0.030576
...     ...               ...         ...
2647  56035  LinearRegression    0.000002
2648  56037  LinearRegression    0.142495
2649  56041  LinearRegression    0.020936
2650  56043  LinearRegression    0.393488
2651  56045  LinearRegression    0.221829

[2652 rows x 3 columns]
